In [1]:
import os
import warnings
from enum import Enum
from typing import Optional, Any

from langchain.chat_models.gigachat import GigaChat
from langchain.chains import ConversationChain
from langchain.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryMemory,
)

warnings.filterwarnings("ignore")

In [2]:
from dotenv import load_dotenv

# Load environment variables
load_dotenv("../.env")

True

# 1. Memory

Define your own class implementing a simple LLM-based chatbot. You need to use at least three memory types (langchain.memory), which are set as one argument in the ```init``` definition. If the memory type has any parameters, you also need to define them as arguments in the ```init``` definition. You also need to define a ```run``` method implementing the main conversation loop, and a ```print_memory``` method to print out what exactly the memory consists of.

In [4]:
class MemoryType(Enum):
    SIMPLE = "simple"
    WINDOW = "window"
    SUMMARY = "summary"

    @staticmethod
    def try_parse(value: str) -> Optional["MemoryType"]:
        try:
            return MemoryType(value.lower())
        except ValueError:
            return None


class SimpleChatBot:
    def __init__(self, llm: GigaChat, memory_type: str, memory_window_size: Optional[int] = None):
        self.llm = llm
        self.memory_type = MemoryType.try_parse(memory_type)

        if not self.memory_type:
            raise ValueError(f"Invalid memory type: {memory_type}")

        self.memory = self._initialize_memory(memory_window_size)
        self.conversation_chain = ConversationChain(llm=self.llm, memory=self.memory)

    def _initialize_memory(self, memory_window_size: Optional[int]) -> Any:
        if self.memory_type == MemoryType.SIMPLE:
            return ConversationBufferMemory()
        elif self.memory_type == MemoryType.WINDOW:
            if memory_window_size is None:
                raise ValueError("memory_window_size is required for window memory.")

            return ConversationBufferWindowMemory(k=memory_window_size)
        elif self.memory_type == MemoryType.SUMMARY:
            return ConversationSummaryMemory(llm=self.llm)
        else:
            raise ValueError("Unsupported memory type")

    def print_memory(self):
        if hasattr(self.memory, "buffer"):
            print(self.memory.buffer)
        elif hasattr(self.memory, "load_memory_variables"):
            print(self.memory.load_memory_variables({}))
        else:
            print("Memory is not accessible for this type.")

    def run(self):
        print("This is a simple chat bot:")
        while True:
            user_input = input("You: ")
            print(f"User: {user_input}")
            if user_input.lower() in ["exit", "quit"]:
                print("Exiting chat.")
                break

            response = self._respond(user_input)
            print(f"Bot: {response}")

    def _respond(self, user_input: str) -> str:
        return self.conversation_chain.run(input=user_input)

Now let's check how it works with each type of memory

**Summary**

In [23]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)

In [24]:
chat = SimpleChatBot(giga, "summary", memory_window_size=2)
chat.run()

This is a simple chat bot:
User: Расскажи мне про Екатеринбург как туристический город, объём рассказа должен быть до 150 слов
Bot: Конечно! Екатеринбург - это большой город с богатой историей и культурой. Он расположен на Урале, который является важным промышленным регионом России. Город известен своими музеями, театрами и памятниками архитектуры. Одним из самых известных мест является Храм-на-Крови, построенный на месте убийства последнего русского царя Николая II и его семьи. Также стоит посетить Плотинку - место, где начинается река Исеть, и откуда открывается прекрасный вид на город. В Екатеринбурге также есть много парков и скверов, где можно насладиться природой и отдохнуть от городской суеты.
User: exit
Exiting chat.


In [25]:
chat.print_memory()

Екатеринбург - большой город с богатой историей и культурой, расположенный на Урале. Он известен своими музеями, театрами и памятниками архитектуры. Храм-на-Крови и Плотинка - популярные места для посещения. В городе также много парков и скверов.


**Window**

In [28]:
chat = SimpleChatBot(giga, "window", memory_window_size=1)
chat.run()

This is a simple chat bot:
User: Расскажи мне про Екатеринбург как туристический город, объём рассказа должен быть до 150 слов
Bot: Конечно! Екатеринбург - это крупный промышленный город с богатой историей и культурой. Он расположен на востоке России, недалеко от Уральских гор. Город известен своими красивыми парками и скверами, такими как Парк Маяковского и ЦПКиО имени В.В. Маяковского. Также стоит посетить Исторический сквер, где можно увидеть памятники архитектуры XIX века. В Екатеринбурге много музеев, включая Музей изобразительных искусств, который славится своей коллекцией русского искусства. Кроме того, в городе есть несколько театров, включая Театр оперы и балета, который считается одним из лучших в стране. В целом, Екатеринбург - это интересное место для посещения, предлагающее разнообразные возможности для отдыха и развлечений.
User: Расскажи мне про Сыктывкар как туристический город, объём рассказа - 150 слов
Bot: К сожалению, у меня нет информации о Сыктывкаре как о туристи

In [29]:
chat.print_memory()

Human: Расскажи мне про Сыктывкар как туристический город, объём рассказа - 150 слов
AI: К сожалению, у меня нет информации о Сыктывкаре как о туристическом городе. Могу ли я помочь вам с чем-то ещё?


**Simple**

In [30]:
chat = SimpleChatBot(giga, "simple", memory_window_size=2)
chat.run()

This is a simple chat bot:
User: Расскажи мне про Екатеринбург как туристический город, объём рассказа должен быть до 150 слов
Bot: Конечно! Екатеринбург - это город с богатой историей и множеством достопримечательностей. Он расположен на Урале, который является границей между Европой и Азией. Город был основан в 1723 году как заводской поселок для строительства железоделательного завода. Сегодня Екатеринбург - это современный мегаполис с развитой инфраструктурой и множеством интересных мест для посещения. Здесь можно увидеть такие достопримечательности, как Плотинка, Храм-на-Крови, Исторический сквер, Уральский государственный университет, Театр оперы и балета, Музей изобразительных искусств и многие другие. В городе также есть несколько парков и садов, где можно насладиться природой и отдохнуть от городской суеты. Екатеринбург также известен своими фестивалями и мероприятиями, такими как Уральская ночь музыки, День города и другие. Это прекрасный город для туризма, который предлагает

In [31]:
chat.print_memory()

Human: Расскажи мне про Екатеринбург как туристический город, объём рассказа должен быть до 150 слов
AI: Конечно! Екатеринбург - это город с богатой историей и множеством достопримечательностей. Он расположен на Урале, который является границей между Европой и Азией. Город был основан в 1723 году как заводской поселок для строительства железоделательного завода. Сегодня Екатеринбург - это современный мегаполис с развитой инфраструктурой и множеством интересных мест для посещения. Здесь можно увидеть такие достопримечательности, как Плотинка, Храм-на-Крови, Исторический сквер, Уральский государственный университет, Театр оперы и балета, Музей изобразительных искусств и многие другие. В городе также есть несколько парков и садов, где можно насладиться природой и отдохнуть от городской суеты. Екатеринбург также известен своими фестивалями и мероприятиями, такими как Уральская ночь музыки, День города и другие. Это прекрасный город для туризма, который предлагает много интересного для посе

Please make a short report about differences between used memory types

**Report:**
1. ConversationBufferMemory
    - Суть: Хранит всю историю взаимодействия в полном объеме, без ограничения по длине.
    - Отличие: Постоянно добавляет новые сообщения в память, не удаляя старые.
    - Использование: Подходит для сценариев, где важна вся история диалога, например, для глубокого анализа или задач с длительными контекстами.

2. ConversationBufferWindowMemory
    - Суть: Хранит только последние k сообщений (окно).
    - Отличие: Ограничивает количество сообщений, которые бот "помнит", сохраняя только последние пользовательские запросы и ответы бота.
    - Использование: Подходит для краткосрочных диалогов, когда нужно ограничить контекст для повышения производительности или симуляции кратковременной памяти.

3. ConversationSummaryMemory
    - Суть: Создает и поддерживает краткое резюме разговора вместо сохранения всех сообщений.
    - Отличие: Сжимает всю историю в краткие ключевые моменты, сохраняя только суть диалога.
    - Использование: Идеально для длительных разговоров, где нужно сохранить общее понимание без хранения полного объема данных, например, для поддержки долгосрочного контекста без излишней детализации.

# 2. Using tools and agents

## 2.1 Using tools and API

Create your own tool based on the langchain.tools library to interact with a public OpenWeather API. This tool will receive data from the API and return it as a readable result for the user.


OpenWeather API URL: https://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_key}&units=metric

[How to get OpenWeather API key](https://docs.google.com/document/d/1vbi8QKqMZqZoCReIzpmEB_2mHsrbmXPlyGngE3jeDDw/edit)

In [13]:
import os

import requests
from langchain_core.pydantic_v1 import BaseModel, Field

from langchain.tools import tool, Tool
from langchain.agents import create_gigachat_functions_agent, AgentExecutor

In [75]:
openweather_key = os.environ.get("OPENWEATHER_API_KEY")


class GetWeatherResult(BaseModel):
    city: str = Field(description="Name of the city in nominative case (Москва, Ставрополь)")
    weather_description: str = Field(description="Weather description (ясно, дождь)")
    temperature: str = Field(description="Temperature in degrees Celsius (-1, 5)")


few_shot_examples_weather = [
    {
        "request": "Какая погода сейчас в Ставрополе?",
        "params": {"city": "Ставрополь"},
    }
]


@tool(few_shot_examples=few_shot_examples_weather)
def get_weather(
    city: str = Field(description="Name of the city in nominative case (Москва, Ставрополь)"),
) -> GetWeatherResult:
    """Gets weather information by city name"""

    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_key}&units=metric"
    response = requests.get(url, timeout=5)

    if response.status_code != 200:
        return GetWeatherResult(city=city, weather_description="Не удалось получить", temperature=0.0)

    data = response.json()

    weather_description = data["weather"][0]["description"]
    temperature = data["main"]["temp"]

    return GetWeatherResult(city=city, weather_description=weather_description, temperature=temperature)


class OpenWeatherAPITool:
    def __init__(self, llm: GigaChat, tool: Tool):
        agent = create_gigachat_functions_agent(llm, [tool])
        self.agent_executor = AgentExecutor(agent=agent, tools=[tool], verbose=True)

    def run(self, user_input: str) -> dict:
        return self.agent_executor.invoke({"input": user_input})

Let's check it

In [18]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga_pro = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)

In [19]:
openweather_tool = OpenWeatherAPITool(giga_pro, get_weather)
openweather_tool.run("Какая погода сейчас в Москве?")



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'city': 'Москва'}`


https://api.openweathermap.org/data/2.5/weather?q=Москва&appid=178ca7c96f5bf8db0ebd5015f2dd1f36&units=metric
city='Москва' weather_description='overcast clouds' temperature='6.58'В Москве сейчас облачно и температура воздуха составляет 6.58°С.

> Finished chain.


{'input': 'Какая погода сейчас в Москве?',
 'output': 'В Москве сейчас облачно и температура воздуха составляет 6.58°С.'}

## 2.2. Multi agents

Create a multi-agent system where each agent is responsible for a specific task in the travel planning process. For example, one agent is responsible for searching for flights, another for booking hotels, and a third for finding the weather at the destination.

Requirements:

- Use three or more GigaChat-based agents to interact with each other.
- The first agent is responsible for searching for flights (using ```get_url_booking_tickets``` function).
- The second agent is responsible for booking hotels (using ```get_url_booking_hotels``` function).
- The third agent collects weather information for the destination (using a real API, such as OpenWeather). You can use the function from the previous task (for simplify, here you can give a current weather, not a forecast for the specific date)

In [53]:
from datetime import datetime, timedelta
from dateutil import parser
import urllib.parse
import json

In [54]:
def get_geoid(city: str) -> str:
    url_base = "https://suggest-maps.yandex.ru/suggest-geo"
    params = {
        "search_type": "tune",
        "v": "9",
        "results": 1,
        "lang": "ry_RU",
        "callback": "json",
        "part": city,
    }

    r = requests.get(url_base, params=params)
    if not r.ok:
        return ""

    r_text = r.text
    r_json = r_text[5 : len(r_text) - 1]
    res_json = json.loads(r_json)
    res = res_json["results"][0]["geoid"]

    return str(res)

In [55]:
class BookingHotelsResult(BaseModel):
    link: str = Field(description="Only the link to the hotel booking, provided by tool")


few_shot_examples_hotels = [
    {
        "request": "Организуй поездку в Санкт-Петербург на 10 дней с 15.11.2024 - отель, самолет, погода",
        "params": {"date_in_str": "2024-11-15", "date_out_str": "2024-11-25", "city": "Санкт-Петербург"},
    }
]


@tool(few_shot_examples=few_shot_examples_hotels)
def get_url_booking_hotels(
    date_in_str: str = Field(description="Date of check-in to the hotel"),
    date_out_str: str = Field(description="Date of check-out to the hotel"),
    city: str = Field(description="City in which a hotel requires to be booked"),
) -> BookingHotelsResult:
    """The method allows you to get a link to a hotel booking in the specified city on the specified dates."""

    date_in = parser.parse(date_in_str)
    if date_in is None:
        date_in = datetime.now()

    date_out = parser.parse(date_out_str)
    if date_out is None:
        date_out = datetime.now() + timedelta(days=1)

    geoid = get_geoid(city)
    url = "https://travel.yandex.ru/hotels/search/?"
    params = {
        "adults": "2",
        "checkinDate": date_in.strftime("%Y-%m-%d"),
        "checkoutDate": date_out.strftime("%Y-%m-%d"),
        "childrenAges": "0",
        "geoId": geoid,
    }

    url = f"{url}{urllib.parse.urlencode(params)}"

    return BookingHotelsResult(link=url)

In [56]:
class BookingTicketsResult(BaseModel):
    link: str = Field(description="Only the link to the tickets booking, provided by tool")


few_shot_examples_tickets = [
    {
        "request": "Организуй поездку из Москвы в Санкт-Петербург на 10 дней с 15.11.2024 - отель, самолет, погода",
        "params": {
            "date_in_str": "2024-11-15",
            "date_out_str": "2024-11-25",
            "city_from": "Москва",
            "date_out": "Санкт-Петербург",
        },
    }
]


@tool(few_shot_examples=few_shot_examples_tickets)
def get_url_booking_tickets(
    date_in_str: str = Field(description="Date of check-in to the hotel"),
    date_out_str: str = Field(description="Date of check-out to the hotel"),
    city_from: str = Field(description="City from which the addressee originates"),
    city_to: str = Field(description="The city to which the addressee is travelling"),
) -> BookingTicketsResult:
    """The method provides a link to purchase flight tickets between specified cities on specified dates."""

    date_in = parser.parse(date_in_str)
    if date_in is None:
        date_in = datetime.now()

    date_out = parser.parse(date_out_str)
    if date_out is None:
        date_out = datetime.now() + timedelta(days=1)

    fromid = get_geoid(city_from)
    toid = get_geoid(city_to)

    url = "https://travel.yandex.ru/avia/search/result/?"
    params = {
        "adults_seats": "2",
        "fromId": "c" + fromid,
        "klass": "economy",
        "oneway": "2",
        "return_date": date_out.strftime("%Y-%m-%d"),
        "toId": "c" + toid,
        "when": date_in.strftime("%Y-%m-%d"),
    }

    url = f"{url}{urllib.parse.urlencode(params)}"

    return BookingTicketsResult(link=url)

In [76]:
class MultiAgent:
    def __init__(
        self,
        llm: GigaChat,
        agent_function_weather: Tool,
        agent_function_hotels: Tool,
        agent_function_tickets: Tool,
    ):
        self.llm = llm
        self.agent_function_weather = agent_function_weather
        self.agent_function_hotels = agent_function_hotels
        self.agent_function_tickets = agent_function_tickets

        self.hotel_agent = self._create_agent([self.agent_function_hotels])
        self.flight_agent = self._create_agent([self.agent_function_tickets])
        self.weather_agent = self._create_agent([self.agent_function_weather])

    def run(self, user_input: str):
        weather_result = self.weather_agent.invoke({"input": f"Какая погода в городе прибытия `{user_input}`"})
        hotel_result = self.hotel_agent.invoke(
            {"input": f"Предоставь ссылку на бронирование отеля по информации `{user_input}`"}
        )
        flight_result = self.flight_agent.invoke(
            {"input": f"Предоставь ссылку на бронирование билетов по информации  `{user_input}`"}
        )

        return f"{hotel_result['output']}\n{flight_result['output']}\n{weather_result['output']}"

    def _create_agent(self, tools: list[Tool]) -> AgentExecutor:
        agent = create_gigachat_functions_agent(giga_pro, tools)
        return AgentExecutor(agent=agent, tools=tools, verbose=False)

In [77]:
giga_key = os.environ.get("SB_AUTH_DATA")
giga_pro = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)

In [78]:
traveler = MultiAgent(giga_pro, get_weather, get_url_booking_hotels, get_url_booking_tickets)
user_input = "Организуй поездку из Москвы в Санкт-Петербург на 10 дней с 15.11.2024 - отель, самолет, погода"
answer = traveler.run(user_input)
print(answer)

Вот ссылка на бронирование отеля для Вашего путешествия из Москвы в Санкт-Петербург на 10 дней с 15.11.2024: https://travel.yandex.ru/hotels/search/?adults=2&checkinDate=2024-11-15&checkoutDate=2024-11-25&childrenAges=0&geoId=213.
Вот ссылка для бронирования билетов по вашему запросу: https://travel.yandex.ru/avia/search/result/?adults_seats=2&fromId=c213&klass=economy&oneway=2&return_date=2024-11-25&toId=c2&when=2024-11-15.
В Санкт-Петербурге в день Вашего прибытия ожидается легкий дождь, температура воздуха составит 7.25°С.
